# Description
Probable Name: Neris

MD5: bf08e6b02e00d2bc6dd493e93e69872f

SHA1: 5c2ba68d78471ff02adcdab12b2f82db8efe2104

SHA256: 527da5fd4e501765cdd1bccb2f7c5ac76c0b22dfaf7c24e914df4e1cb8029d71

Password of zip file: infected

Duration: 3 hours, 51 minutes and 0 seconds

VirusTotal

HybridAnalysis

RobotHash

In [13]:
import pandas as pd

# Define the path to your file
file_path = "../../data/Neris/Modeling_Neris.csv"

# Load the data
# Since your data appears to be tab-separated, we use sep='\s+' which handles multiple spaces
df = pd.read_csv(
    file_path)
df.drop(["Unnamed: 0","Flows"], axis=1, inplace=True)
df.head()

,index,Duration,Protocol,Flags,Packets,Bytes,Label,Source Port,Destination Port,Bytes per Packet,Packets per Second,Bytes per Second,Is Encrypted Traffic,Common Port Usage
0,0,0.281,TCP,FSPA_,5.0,533.0,LEGITIMATE,2596,80,106.600000,17.793594,1896.797153,0,1
1,1,0.069,TCP,_FSA,2.0,122.0,Botnet,1842,80,61.000000,28.985507,1768.115942,0,1
2,2,0.125,TCP,FSA_,4.0,242.0,Botnet,1040,80,60.500000,32.000000,1936.000000,0,1
3,3,3.605,TCP,SPA_,11.0,6596.0,LEGITIMATE,50369,80,599.636364,3.051318,1829.680999,0,1
4,4,0.001,TCP,S_,1.0,62.0,Botnet,4192,179,62.000000,1000.000000,62000.000000,0,0


# Modeling

## Data Prep

In [14]:
df["Bytes"] = df["Bytes"].astype("float64")
df["Duration"] = df["Duration"].astype("float64")
df["Packets"] = df["Packets"].astype("float64")
df = df.reset_index()

df["Label"] = df["Label"].replace(
    {"LEGITIMATE": "Normal", "Botnet": "Neris"}
)

In [5]:
def save_as_pickle(path, variable):
    with open(path, "wb") as file:
        pickle.dump(variable, file)

In [7]:
import pickle
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

columns_to_encode = ["Protocol", "Flags", "Label"]

for column in columns_to_encode:
    # Fit label encoder and return encoded labels
    path = "../../variables/Neris/" + column + "_Encoder.pkl"
    df[column] = encoder.fit_transform(df[column])
    save_as_pickle(path, encoder)

In [8]:
from sklearn.preprocessing import StandardScaler
# Separate the 'Status' column
label_column = df["Label"]
dropped_df = df.drop("Label", axis=1)

# Initialize and apply MinMaxScaler to the dropped_df
scaler = StandardScaler()
scaled_dropped_df = scaler.fit_transform(dropped_df.to_numpy())

# Create a DataFrame from the scaled dropped_df
df = pd.DataFrame(scaled_dropped_df, columns=dropped_df.columns)

# Reattach the 'Status' column
df["Label"] = label_column

save_as_pickle(r"../../variables/Neris/scaler.pkl", scaler)

## Creating Model

In [11]:
from sklearn.linear_model import LogisticRegression
import prisma_ml as prisma

max_iter = 5000
logreg_model = LogisticRegression(max_iter=max_iter)

X = df.drop("Label", axis=1)
y = df["Label"]

test_size = 0.3
iterations = 30

logreg_normal = prisma.MachineLearning()
(
    logreg_normal_metrics,
    avg_logreg_normal_metrics,
    logreg_normal_model,
) = logreg_normal.evaluate_model(X, y, test_size, iterations, logreg_model)

Evaluating model: 100%|██████████| 30/30 [01:42<00:00,  3.40s/it]


In [12]:
avg_logreg_normal_metrics

{'Accuracy': 0.842, 'Precision': 0.842, 'Recall': 0.842, 'F1 Score': 0.841}

In [ ]:
save_as_pickle(r"../../models/NerisLogreg.pkl", logreg_normal_model)